In [1]:
import pandas as pd
import pickle
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import random
import copy

print(f"current wd {os.getcwd()}")

current wd /Users/carlos/Documents/Peccala/deepL_tests


In [5]:
import os
import numpy as np
import logging
import sys

# Define the directory containing the .npy files
directory_path = "data_deep_learning/series_data"

tst_perc = 0.05
val_perc = 0.1

seq_length = 32
d_model = 80
n_head = 2
max_len = 3000
ffn_hidden = 60
n_layers = 2
n_classes = 1
drop_prob = 0.4
details = False
epochs = 50
batch_size = 128
patience = 14
model = None

'''logging.basicConfig(
    filename="progress.log",  # File where logs will be saved
    level=logging.INFO,       # Log level (e.g., DEBUG, INFO, WARNING, ERROR)
    format="%(asctime)s - %(levelname)s - %(message)s",  # Log format
    datefmt="%Y-%m-%d %H:%M:%S"  # Timestamp format
    import sys

# Redirect stdout to a file

)'''

if True : #with open("progress.log", "w") as f:
    #sys.stdout = f

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    for file_name in os.listdir(directory_path):
        print("Starting the process...")
        key = file_name.split('.')[0].replace("&","/")
        if file_name.endswith(".npy") and "DOGE" in key:  
            #key = file_name.split('.')[0].replace("&","/")
            print(f"Processing series {key}...")

            data = np.load(directory_path+"/"+file_name)
            X=data[:,:-1]
            y=data[:,-1]
            del data
            for i in range(X.shape[0]):
                if not np.isnan(sum(X[i, 72:])):
                    print(f" not nan's at {i}")
                    l_ind = i
                    break
            for i in range(l_ind, X.shape[0]):
                if np.isnan(sum(X[i, 72:])):
                    print(f" at {key} NaNs at {i}")
            if l_ind > X.shape[0] - 6000:
                X=X[l_ind:, :]
                y=y[l_ind:]
            else:    
                X=X[-6000:, :]
                y=y[-6000:]
            
            n_X = norm_mts_with_window(X, window_size=30)    
            n_X = create_seq_from_ts(n_X, seq_length)
            print(f"norm and turning into seqences complete.")
            n_y=y[seq_length-1:]    
            
            ind_tst = int(n_X.shape[0] * tst_perc)
            ind_val = int(n_X.shape[0] * val_perc)
            
            X_tr, X_val, X_tst = n_X[: -(ind_tst + ind_val),:,:], n_X[-(ind_tst + ind_val):-ind_tst,:,:], n_X[-ind_tst:,:,:] 
            y_tr, y_val, y_tst = n_y[: -(ind_tst + ind_val)], n_y[-(ind_tst + ind_val):-ind_tst], n_y[-ind_tst:] 

            X_tr_t = torch.tensor(X_tr, dtype=torch.float32).to(device)
            y_tr_t = torch.tensor(y_tr, dtype=torch.float32).to(device)
            X_val_t = torch.tensor(X_val, dtype=torch.float32).to(device)
            y_val_t = torch.tensor(y_val, dtype=torch.float32).to(device)
            X_tst_t = torch.tensor(X_tst, dtype=torch.float32).to(device)
            y_tst_t = torch.tensor(y_tst, dtype=torch.float32).to(device)

            #scale targets to lie between 1 and 0
            y_tr_t = (y_tr_t +1) / 2
            y_val_t = (y_val_t +1) / 2
            y_tst_t = (y_tst_t +1) / 2
            
            SEED = 77
            random.seed(SEED)
            np.random.seed(SEED)
            torch.manual_seed(SEED)
            if torch.cuda.is_available():
                torch.cuda.manual_seed(SEED)
                torch.cuda.manual_seed_all(SEED)
                torch.backends.cudnn.deterministic = True
                torch.backends.cudnn.benchmark = False


            
            best_v_acc = 0
            n_noimprov = 0
            best_model_params = None

            y_tr_tensor = torch.tensor(y_tr_t, dtype=torch.long)
            y_val_tensor = torch.tensor(y_val_t, dtype=torch.long)


            # Prepare data loaders
            train_dataset = TensorDataset(X_tr_t, y_tr_tensor)
            val_dataset = TensorDataset(X_val_t, y_val_tensor)
            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=batch_size)

            if not model:
                print(f"Creating the model.")   
                model = Transformer(
                    device=device,
                    d_model=d_model,
                    n_head=n_head,
                    max_len=max_len,
                    seq_len=seq_length,
                    ffn_hidden=ffn_hidden,
                    n_layers=n_layers,
                    drop_prob=drop_prob,
                    details=details
                ).to(device)
            pos_weight = len(y_tr_t) / sum(y_tr_t)  # Assumes y_tr_t is binary {0, 1}
            neg_weight = len(y_tr_t) / (len(y_tr_t) - sum(y_tr_t))

            # Use a tensor to represent positive and negative weights
            class_weight = torch.tensor([neg_weight, pos_weight]).to(device)

            # Update criterion
            criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
            optimizer = optim.Adam(model.parameters(), lr=1e-3)
            print(f"------------------for series {key} training loop")
            # Training loop
            for epoch in range(epochs):
                model.train()
                train_loss = 0.0
                correct = 0
                total = 0

                for X_batch, y_batch in train_loader:
                    X_batch, y_batch = X_batch.to(device), y_batch.to(device).unsqueeze(1)

                    # Ensure y_batch is of the correct type
                    y_batch = y_batch.float()
                    outputs = model(X_batch)
                    loss = criterion(outputs, y_batch)

                    # Backward pass and optimization
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                    train_loss += loss.item() * X_batch.size(0)

                    # Compute predictions and accuracy
                    probabilities = torch.sigmoid(outputs)
                    predicted = (probabilities > 0.5).float()
                    correct += (predicted == y_batch).sum().item()
                    total += y_batch.size(0)

                train_loss /= len(train_loader.dataset)
                train_accuracy = 100 * correct / total
                
                # Validation loop
                model.eval()
                val_loss = 0.0
                correct = 0
                total = 0
                with torch.no_grad():
                    for X_batch, y_batch in val_loader:
                        X_batch, y_batch = X_batch.to(device), y_batch.to(device).unsqueeze(1)
                        y_batch = y_batch.float()

                        outputs = model(X_batch)
                        loss = criterion(outputs, y_batch)

                        val_loss += loss.item() * X_batch.size(0)
                        probabilities = torch.sigmoid(outputs)
                        predicted = (probabilities > 0.5).float()
                        correct += (predicted == y_batch).sum().item()
                        total += y_batch.size(0)

                val_loss /= len(val_loader.dataset)
                val_accuracy = 100 * correct / total
                
                if train_accuracy > 90:
                    if val_accuracy > best_v_acc:
                        best_v_acc = val_accuracy
                        best_model_params = copy.deepcopy(model.state_dict())
                        n_noimprov = 0
                    else:
                        n_noimprov +=1
                            
                if n_noimprov > patience and best_model_params: #best_model_params is not None
                    print(f"Early stopping triggered, best acc on val data {best_v_acc}")   
                    break 

                print(f"Epoch {epoch + 1}/{epochs}, "
                    f"Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, "
                    f"Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%")

            print("Training complete.")
        
        
sys.stdout = sys.__stdout__
print("Progress log has been written to progress.log")        
        

/var/folders/0f/11hy697j7mv487fs334bjwvr0000gp/T/ipykernel_39319/3524868131.py:107: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_tr_tensor = torch.tensor(y_tr_t, dtype=torch.long)
/var/folders/0f/11hy697j7mv487fs334bjwvr0000gp/T/ipykernel_39319/3524868131.py:108: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_val_tensor = torch.tensor(y_val_t, dtype=torch.long)


In [1]:
#X=X[-3000:,:]
#y=y[-3000:]
#print(f"x shape {X.shape}, y s {y.shape}")
#sys.stdout = sys.__stdout__
print("hi")


hi


In [2]:
import numpy as np

def norm_mts_with_window(data, window_size=30):
    """
    Normalize a multivariate time series so that each component of the vector has
    a standard deviation of 1 within a sliding window lokking back in time.
    If window size is 1 does nothing.

    Returns np.array normalized time series data with the same shape as input.
    """
    t_steps, num_features = data.shape
    normalized_data = np.zeros_like(data)

    for t in range(t_steps):
        start_i = max(0, t - window_size + 1)
        end_i = t + 1
        window = data[start_i:end_i]
        std = np.std(window, axis=0)
        mean = np.mean(window, axis=0)
        std[std == 0] = 1
        normalized_data[t] = (data[t])/ std

    return normalized_data


#function to create tensor with seg_length of backward steps
def create_seq_from_ts(data, seq_length):
    """
    Function that returns the vectors with corresponding seq_length,
    of observations in the past.

    Output should have shape (data.shape[0] - seq_length, seq_length, num_features)

    """
    sequences = []

    for i in range(len(data) - seq_length + 1):
        # Extract the sequence of features
        sequence = data[i:i+seq_length]
        sequences.append(sequence)

    sequences = np.array(sequences)

    return sequences

# Normalize with a sliding window of size 30
'''n_X = norm_mts_with_window(X, window_size=30)
print("Normalized data shape:", n_X.shape)

seq_length = 32

n_X = create_seq_from_ts(n_X, seq_length)
n_y=y[seq_length-1:]
print("X shape", n_X.shape)
print("y shape ", n_y.shape)'''




'n_X = norm_mts_with_window(X, window_size=30)\nprint("Normalized data shape:", n_X.shape)\n\nseq_length = 32\n\nn_X = create_seq_from_ts(n_X, seq_length)\nn_y=y[seq_length-1:]\nprint("X shape", n_X.shape)\nprint("y shape ", n_y.shape)'

In [ ]:
#cut the data to save RAM ??
sequences = sequences[-3000:,:,:]
print(f"Sequences shape {sequences.shape}")
del n_data
del data
target_btc_f=target_btc_f[-5000:]
print(f"Target shape {target_btc_f.shape}")

In [3]:
import torch

tst_perc = 0.05
val_perc = 0.1
ind_tst = int(n_X.shape[0] * tst_perc)
ind_val = int(n_X.shape[0] * val_perc)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#split train, val, test
X_tr, X_val, X_tst = n_X[: -(ind_tst + ind_val),:,:], n_X[-(ind_tst + ind_val):-ind_val,:,:], n_X[-ind_val:,:,:] 
y_tr, y_val, y_tst = n_y[: -(ind_tst + ind_val)], n_y[-(ind_tst + ind_val):-ind_val], n_y[-ind_val:] 

X_tr_t = torch.tensor(X_tr, dtype=torch.float32).to(device)
y_tr_t = torch.tensor(y_tr, dtype=torch.float32).to(device)
X_val_t = torch.tensor(X_val, dtype=torch.float32).to(device)
y_val_t = torch.tensor(y_val, dtype=torch.float32).to(device)
X_tst_t = torch.tensor(X_tst, dtype=torch.float32).to(device)
y_tst_t = torch.tensor(y_tst, dtype=torch.float32).to(device)

#scale targets to lie between 1 and 0
y_tr_t = (y_tr_t +1) / 2
y_val_t = (y_val_t +1) / 2
y_tst_t = (y_tst_t +1) / 2

# Print shapes for verification
print(f"X_tr_tensor shape: {X_tr_t.shape}")
print(f"y_tr_tensor shape: {y_tr_t.shape}")
print(f"X_tst_tensor shape: {X_tst_t.shape}")
print(f"y_tst_tensor shape: {y_tst_t.shape}")
print(f"y_val_tensor: {y_tr_t.shape}")

NameError: name 'n_X' is not defined

In [3]:
import torch
from torch import nn
import math


class Transformer(nn.Module):

    def __init__(self,device, d_model=100, n_head=4, max_len=500, seq_len=30,
                 ffn_hidden=128, n_layers=4, drop_prob=0.1, details =False):
        super().__init__()
        self.device = device
        self.details = details
        self.encoder_input_layer = nn.Linear(
            in_features=145,
            out_features=d_model
            )

        self.pos_emb = PostionalEncoding( max_seq_len=max_len,batch_first=False, d_model=d_model, dropout=0.1) #try different values of drupout?
        self.encoder = Encoder(d_model=d_model,
                               n_head=n_head,
                               ffn_hidden=ffn_hidden,
                               drop_prob=drop_prob,
                               n_layers=n_layers,
                               details=details,
                               device=device)
        self.classHead = ClassificationHead(seq_len=seq_len,d_model=d_model,details=details,n_classes=1)

    def forward(self, src ):
        if self.details: print('before input layer: '+ str(src.size()) )
        src= self.encoder_input_layer(src)
        if self.details: print('after input layer: '+ str(src.size()) )
        src= self.pos_emb(src)
        if self.details: print('after pos_emb: '+ str(src.size()) )
        enc_src = self.encoder(src)
        cls_res = self.classHead(enc_src)
        if self.details: print('after cls_res: '+ str(cls_res.size()) )
        return cls_res


class PostionalEncoding(nn.Module):
    def __init__(
        self,
        dropout: float=0.1,
        max_seq_len: int=5000,
        d_model: int=512,
        batch_first: bool=False    ):
        super().__init__()

        self.d_model = d_model
        self.dropout = nn.Dropout(p=dropout)
        self.batch_first = batch_first
        self.x_dim = 1 if batch_first else 0
        position = torch.arange(max_seq_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_seq_len, 1, d_model)

        pe[:, 0, 0::2] = torch.sin(position * div_term)

        pe[:, 0, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(self.x_dim)]

        x = self.dropout(x)
        return x


class EncoderLayer(nn.Module):

    def __init__(self, d_model, ffn_hidden, n_head, drop_prob,details):
        super(EncoderLayer, self).__init__()
        self.attention = MultiHeadAttention(d_model=d_model, n_head=n_head, details=details)
        self.norm1 = LayerNorm(d_model=d_model)
        self.dropout1 = nn.Dropout(p=drop_prob)
        self.details = details
        self.ffn = PositionwiseFeedForward(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
        self.norm2 = LayerNorm(d_model=d_model)
        self.dropout2 = nn.Dropout(p=drop_prob)

    def forward(self, x):
        # 1. compute self attention
        _x = x
        x = self.attention(q=x, k=x, v=x )

        if self.details: print('in encoder layer : '+ str(x.size()))
        # 2. add and norm
        x = self.dropout1(x)
        x = self.norm1(x + _x)

        if self.details: print('in encoder after norm layer : '+ str(x.size()))
        # 3. positionwise feed forward network
        _x = x
        x = self.ffn(x)

        if self.details: print('in encoder after ffn : '+ str(x.size()))
        # 4. add and norm
        x = self.dropout2(x)
        x = self.norm2(x + _x)
        return x

class Encoder(nn.Module):

    def __init__(self, d_model, ffn_hidden, n_head, n_layers,
                 drop_prob, details, device):
        super().__init__()


        self.layers = nn.ModuleList([EncoderLayer(d_model=d_model,
                                                  ffn_hidden=ffn_hidden,
                                                  n_head=n_head
                                                  ,details=details,
                                                  drop_prob=drop_prob)
                                     for _ in range(n_layers)])

    def forward(self, x ):
        for layer in self.layers:
            x = layer(x)
        return x


class ClassificationHead(nn.Module):
    def __init__(self,d_model, seq_len , details, n_classes: int = 1):
      super().__init__()
      self.norm = nn.LayerNorm(d_model)
      self.details = details
      #self.flatten = nn.Flatten()
      self.seq = nn.Sequential( nn.Flatten() , nn.Linear(d_model * seq_len , 512) ,nn.ReLU(),nn.Linear(512, 256)
                               ,nn.ReLU(),nn.Linear(256, 128),nn.ReLU(),nn.Linear(128, n_classes))

    def forward(self,x):

      if self.details:  print('in classification head : '+ str(x.size()))
      x= self.norm(x)
      #x= self.flatten(x)
      x= self.seq(x)
      if self.details: print('in classification head after seq: '+ str(x.size()))
      return x
  
class MultiHeadAttention(nn.Module):

    def __init__(self, d_model, n_head, details):
        super(MultiHeadAttention, self).__init__()
        self.n_head = n_head
        self.attention = ScaleDotProductAttention( details=details)
        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)
        self.w_concat = nn.Linear(d_model, d_model)
        self.details = details

    def forward(self, q, k, v ):
        # 1. dot product with weight matrices

        q, k, v = self.w_q(q), self.w_k(k), self.w_v(v)

        if self.details: print('in Multi Head Attention Q,K,V: '+ str(q.size()))
        # 2. split tensor by number of heads
        q, k, v = self.split(q), self.split(k), self.split(v)

        if self.details: print('in splitted Multi Head Attention Q,K,V: '+ str(q.size()))
        # 3. do scale dot product to compute similarity
        out, attention = self.attention(q, k, v )

        if self.details: print('in Multi Head Attention, score value size: '+ str(out.size()))
        # 4. concat and pass to linear layer
        out = self.concat(out)
        out = self.w_concat(out)

        # 5. visualize attention map
        # TODO : we should implement visualization

        if self.details: print('in Multi Head Attention, score value size after concat : '+ str(out.size()))
        return out

    def split(self, tensor):
        """
        split tensor by number of head

        :param tensor: [batch_size, length, d_model]
        :return: [batch_size, head, length, d_tensor]
        """
        batch_size, length, d_model = tensor.size()
        d_tensor = d_model // self.n_head
        tensor = tensor.view(batch_size, length, self.n_head, d_tensor).transpose(1, 2)
        # it is similar with group convolution (split by number of heads)

        return tensor

    def concat(self, tensor):
        """
        inverse function of self.split(tensor : torch.Tensor)

        :param tensor: [batch_size, head, length, d_tensor]
        :return: [batch_size, length, d_model]
        """
        batch_size, head, length, d_tensor = tensor.size()
        d_model = head * d_tensor

        tensor = tensor.transpose(1, 2).contiguous().view(batch_size, length, d_model)
        return tensor


class ScaleDotProductAttention(nn.Module):
    """
    compute scale dot product attention

    Query : given sentence that we focused on (decoder)
    Key : every sentence to check relationship with Qeury(encoder)
    Value : every sentence same with Key (encoder)
    """

    def __init__(self, details):
        super(ScaleDotProductAttention, self).__init__()
        self.softmax = nn.Softmax(dim=-1)
        self.details = details
    def forward(self, q, k, v ,e=1e-12):
        # input is 4 dimension tensor
        # [batch_size, head, length, d_tensor]
        batch_size, head, length, d_tensor = k.size()

        # 1. dot product Query with Key^T to compute similarity
        k_t = k.transpose(2, 3)  # transpose

        if self.details: print('in Scale Dot Product, k_t size: '+ str(k_t.size()))
        score = (q @ k_t) / math.sqrt(d_tensor)  # scaled dot product


        if self.details: print('in Scale Dot Product, score size: '+ str(score.size()))
        # 3. pass them softmax to make [0, 1] range
        score = self.softmax(score)

        if self.details: print('in Scale Dot Product, score size after softmax : '+ str(score.size()))

        if self.details: print('in Scale Dot Product, v size: '+ str(v.size()))
        # 4. multiply with Value
        v = score @ v

        if self.details: print('in Scale Dot Product, v size after matmul: '+ str(v.size()))
        return v, score

class PositionwiseFeedForward(nn.Module):

    def __init__(self, d_model, hidden, drop_prob=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, hidden)
        self.linear2 = nn.Linear(hidden, d_model)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=drop_prob)

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        return x

class LayerNorm(nn.Module):
    def __init__(self, d_model, eps=1e-12):
        super(LayerNorm, self).__init__()
        self.gamma = nn.Parameter(torch.ones(d_model))
        self.beta = nn.Parameter(torch.zeros(d_model))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        var = x.var(-1, unbiased=False, keepdim=True)
        # '-1' means last dimension.

        out = (x - mean) / torch.sqrt(var + self.eps)
        out = self.gamma * out + self.beta
        return out
  


In [11]:
seq_length = 32
d_model = 60
n_head = 2
max_len = 3000
ffn_hidden = 60
n_layers = 2
n_classes = 1
drop_prob = 0.4
details = False
epochs = 50
batch_size = 32
patience = 14
model = Transformer(
                    device="cpu",
                    d_model=d_model,
                    n_head=n_head,
                    max_len=max_len,
                    seq_len=seq_length,
                    ffn_hidden=ffn_hidden,
                    n_layers=n_layers,
                    drop_prob=drop_prob,
                    details=details
                )
sum(p.numel() for p in model.parameters() if p.requires_grad)

1201125

In [10]:
3000*145

435000

In [52]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import random
import copy

# Define constants
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

SEED = 77
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


d_model = 80
n_head = 2
max_len = 3000
seq_len = 32
ffn_hidden = 60
n_layers = 2
n_classes = 1
drop_prob = 0.4
details = False
epochs = 50
batch_size = 32
patience = 14
best_v_acc = 0
n_noimprov = 0
best_model_params = None

y_tr_tensor = torch.tensor(y_tr_t, dtype=torch.long)
y_val_tensor = torch.tensor(y_val_t, dtype=torch.long)


# Prepare data loaders
train_dataset = TensorDataset(X_tr_t, y_tr_tensor)
val_dataset = TensorDataset(X_val_t, y_val_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Initialize the model
model = Transformer(
    device=device,
    d_model=d_model,
    n_head=n_head,
    max_len=max_len,
    seq_len=seq_len,
    ffn_hidden=ffn_hidden,
    n_layers=n_layers,
    drop_prob=drop_prob,
    details=details
).to(device)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training loop
for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device).unsqueeze(1)

        # Ensure y_batch is of the correct type
        y_batch = y_batch.float()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * X_batch.size(0)

        # Compute predictions and accuracy
        probabilities = torch.sigmoid(outputs)
        predicted = (probabilities > 0.5).float()
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)

    train_loss /= len(train_loader.dataset)
    train_accuracy = 100 * correct / total

    # Validation loop
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device).unsqueeze(1)
            y_batch = y_batch.float()

            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)

            val_loss += loss.item() * X_batch.size(0)
            probabilities = torch.sigmoid(outputs)
            predicted = (probabilities > 0.5).float()
            correct += (predicted == y_batch).sum().item()
            total += y_batch.size(0)

    val_loss /= len(val_loader.dataset)
    val_accuracy = 100 * correct / total
    
    if train_accuracy > 90:
        if val_accuracy > best_v_acc:
            best_v_acc = val_accuracy
            best_model_params = copy.deepcopy(model.state_dict())
            n_noimprov = 0
        else:
            n_noimprov +=1
                
    if n_noimprov > patience and best_model_params: #best_model_params is not None
        print(f"Early stopping triggered, best acc on val data {best_v_acc}")   
        break 

    print(f"Epoch {epoch + 1}/{epochs}, "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%")

print("Training complete.")

/var/folders/0f/11hy697j7mv487fs334bjwvr0000gp/T/ipykernel_12909/451683020.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_tr_tensor = torch.tensor(y_tr_t, dtype=torch.long)
/var/folders/0f/11hy697j7mv487fs334bjwvr0000gp/T/ipykernel_12909/451683020.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_val_tensor = torch.tensor(y_val_t, dtype=torch.long)


Epoch 1/50, Train Loss: 0.6167, Train Acc: 65.62%, Val Loss: 0.7604, Val Acc: 53.38%
Epoch 2/50, Train Loss: 0.4615, Train Acc: 79.92%, Val Loss: 0.6742, Val Acc: 60.14%
Epoch 3/50, Train Loss: 0.3813, Train Acc: 83.80%, Val Loss: 0.8470, Val Acc: 68.24%
Epoch 4/50, Train Loss: 0.3299, Train Acc: 86.69%, Val Loss: 0.6418, Val Acc: 73.65%
Epoch 5/50, Train Loss: 0.3010, Train Acc: 87.76%, Val Loss: 1.1079, Val Acc: 56.76%
Epoch 6/50, Train Loss: 0.2964, Train Acc: 87.96%, Val Loss: 1.0577, Val Acc: 63.51%
Epoch 7/50, Train Loss: 0.2881, Train Acc: 88.99%, Val Loss: 0.7835, Val Acc: 68.24%
Epoch 8/50, Train Loss: 0.2389, Train Acc: 91.17%, Val Loss: 1.1693, Val Acc: 54.73%
Epoch 9/50, Train Loss: 0.2215, Train Acc: 91.01%, Val Loss: 0.9616, Val Acc: 62.84%
Epoch 10/50, Train Loss: 0.2071, Train Acc: 91.49%, Val Loss: 0.7692, Val Acc: 66.89%
Epoch 11/50, Train Loss: 0.1873, Train Acc: 92.67%, Val Loss: 0.8613, Val Acc: 68.24%
Epoch 12/50, Train Loss: 0.1818, Train Acc: 92.95%, Val Loss: 1

In [53]:
model.load_state_dict(best_model_params)
model.eval()
probabilities = torch.sigmoid(model(X_tst_t))
predicted = (probabilities > 0.5).float()

In [54]:
p=predicted.squeeze()

In [55]:
(p == y_tst_t).sum()

tensor(173)

In [ ]:
##to adjust for imbalanced classes
pos_weight = len(y_tr_t) / sum(y_tr_t)  # Assumes y_tr_t is binary {0, 1}
neg_weight = len(y_tr_t) / (len(y_tr_t) - sum(y_tr_t))

# Use a tensor to represent positive and negative weights
class_weight = torch.tensor([neg_weight, pos_weight]).to(device)

# Update criterion
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

torch.Size([296])

In [2]:
l=['DUSK/USDT', 'XMR/USDT', 'MATIC/USDT', 'HOT/USDT', 'DOGE/USDT', 'BAND/USDT', 'LINK/USDT', 'REN/USDT', 'WAVES/USDT', 'CVC/USDT', 'ZRX/USDT', 'IOTX/USDT', 'ONT/USDT', 'TOMO/USDT', 'CELR/USDT', 'BTC/USDT', 'DENT/USDT', 'RVN/USDT', 'THETA/USDT', 'FTT/USDT', 'IOST/USDT', 'NULS/USDT', 'ANKR/USDT', 'ETC/USDT', 'ATOM/USDT', 'ZEC/USDT', 'ETH/USDT', 'NEO/USDT', 'VET/USDT', 'ONG/USDT', 'EOS/USDT', 'FTM/USDT', 'OGN/USDT', 'XLM/USDT', 'CHZ/USDT', 'ENJ/USDT', 'IOTA/USDT', 'NKN/USDT', 'ADA/USDT', 'STX/USDT', 'QTUM/USDT', 'FET/USDT', 'ICX/USDT', 'HBAR/USDT', 'ARPA/USDT', 'BNB/USDT', 'BAT/USDT', 'BCH/USDT', 'MTL/USDT', 'OMG/USDT', 'COS/USDT', 'KAVA/USDT', 'XRP/USDT', 'LTC/USDT', 'DASH/USDT', 'ONE/USDT', 'TRX/USDT', 'ALGO/USDT', 'RLC/USDT', 'ZIL/USDT', 'XTZ/USDT', 'KEY/USDT']
len(l)

62

In [7]:
import os
directory_path = "data_deep_learning/series_data"

tst_perc = 0.05
val_perc = 0.1

seq_length = 32
d_model = 80
n_head = 2
max_len = 3000
ffn_hidden = 60
n_layers = 2
n_classes = 1
drop_prob = 0.4
details = False
epochs = 50
batch_size = 32
patience = 14
model = None
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Transformer(device=device,
                        d_model=d_model,
                        n_head=n_head,
                        max_len=max_len,
                        seq_len=seq_length,
                        ffn_hidden=ffn_hidden,
                        n_layers=n_layers,
                        drop_prob=drop_prob,
                        details=details
                    ).to(device) 
model.load_state_dict(torch.load("saved_model_params_6.pth"))




for file_name in os.listdir(directory_path):
    print("Starting the process...")
    if file_name.endswith(".npy"):  
        key = file_name.split('.')[0].replace("&","/")
        print(f"Processing series {key}...")
        data = np.load(directory_path+"/"+file_name)
        X=data[:,:-1]
        y=data[:,-1]
        X=X[-3000:, 1:]
        y=y[-3000:]
        
        n_X = norm_mts_with_window(X, window_size=30)    
        n_X = create_seq_from_ts(n_X, seq_length)
        print(f"norm and turning into sequences complete.")
        n_y=y[seq_length-1:]    
        break    

Starting the process...
Processing series DUSK/USDT...
norm and turning into sequences complete.


In [9]:

ind_val = int(n_X.shape[0] * 0.05)

X_tst= n_X[-ind_val:,:,:]
y_tst = n_y[-ind_val:] 

X_tst_t = torch.tensor(X_tst, dtype=torch.float32).to(device)
y_tst_t = torch.tensor(y_tst, dtype=torch.float32).to(device)

y_tst_t = (y_tst_t +1) / 2

model.eval()
with torch.no_grad():
    outputs = model(X_tst_t)
    probabilities = torch.sigmoid(outputs)
    predicted = (probabilities > 0.5).float()

In [12]:
p_pred = predicted.squeeze()
(y_tst_t == p_pred).sum()

tensor(52)

In [14]:
y_tst_t.shape[0]

148

In [1]:
resulsts="""accuracy of model at DUSK/USDT --> 56.75675582885742
accuracy of model at XMR/USDT --> 71.62162017822266
accuracy of model at MATIC/USDT --> 75.67567443847656
accuracy of model at HOT/USDT --> 37.16216278076172
accuracy of model at DOGE/USDT --> 51.35135269165039
accuracy of model at BAND/USDT --> 60.81081008911133
accuracy of model at LINK/USDT --> 33.783782958984375
accuracy of model at REN/USDT --> 64.8648681640625
accuracy of model at WAVES/USDT --> 25.0
accuracy of model at CVC/USDT --> 39.18918991088867
accuracy of model at ZRX/USDT --> 31.08108139038086
accuracy of model at IOTX/USDT --> 45.27027130126953
accuracy of model at ONT/USDT --> 35.81081008911133
accuracy of model at TOMO/USDT --> 72.97297668457031
accuracy of model at CELR/USDT --> 53.378379821777344
accuracy of model at BTC/USDT --> 73.64865112304688
accuracy of model at DENT/USDT --> 53.378379821777344
accuracy of model at RVN/USDT --> 45.945945739746094
accuracy of model at THETA/USDT --> 27.027027130126953
accuracy of model at FTT/USDT --> 39.864864349365234
accuracy of model at IOST/USDT --> 32.43243408203125
accuracy of model at NULS/USDT --> 44.5945930480957
accuracy of model at ANKR/USDT --> 33.10810852050781
accuracy of model at ETC/USDT --> 55.4054069519043
accuracy of model at ATOM/USDT --> 44.5945930480957
accuracy of model at ZEC/USDT --> 76.35134887695312
accuracy of model at ETH/USDT --> 20.945945739746094
accuracy of model at NEO/USDT --> 36.486488342285156
accuracy of model at VET/USDT --> 43.91891860961914
accuracy of model at ONG/USDT --> 24.324323654174805
accuracy of model at EOS/USDT --> 36.486488342285156
accuracy of model at FTM/USDT --> 60.135135650634766
accuracy of model at OGN/USDT --> 62.16216278076172
accuracy of model at XLM/USDT --> 40.5405387878418
accuracy of model at CHZ/USDT --> 45.27027130126953
accuracy of model at ENJ/USDT --> 38.513511657714844
accuracy of model at IOTA/USDT --> 36.486488342285156
accuracy of model at NKN/USDT --> 52.702701568603516
accuracy of model at ADA/USDT --> 47.97297286987305
accuracy of model at STX/USDT --> 58.10810852050781
accuracy of model at QTUM/USDT --> 39.864864349365234
accuracy of model at FET/USDT --> 72.97297668457031
accuracy of model at ICX/USDT --> 27.027027130126953
accuracy of model at HBAR/USDT --> 42.56756591796875
accuracy of model at ARPA/USDT --> 26.351350784301758
accuracy of model at BNB/USDT --> 68.9189224243164
accuracy of model at BAT/USDT --> 34.4594612121582
accuracy of model at BCH/USDT --> 41.89189147949219
accuracy of model at MTL/USDT --> 40.5405387878418
accuracy of model at OMG/USDT --> 27.70270347595215
accuracy of model at COS/USDT --> 41.216217041015625
accuracy of model at KAVA/USDT --> 30.405405044555664
accuracy of model at XRP/USDT --> 49.32432556152344
accuracy of model at LTC/USDT --> 77.70270538330078
accuracy of model at DASH/USDT --> 26.351350784301758
accuracy of model at ONE/USDT --> 45.27027130126953
accuracy of model at TRX/USDT --> 45.945945739746094
accuracy of model at ALGO/USDT --> 48.64864730834961
accuracy of model at RLC/USDT --> 22.972972869873047
accuracy of model at ZIL/USDT --> 25.0
accuracy of model at XTZ/USDT --> 53.378379821777344
accuracy of model at KEY/USDT --> 56.75675582885742
"""
m=resulsts.split(" --> ")
scores=[a.split("acc")[0].replace('\n','') for a in m][1:]

In [2]:
sum([float(s) for s in scores])/len(scores)

45.65170038900068